<h1>F20BC Coursework</h1>
<p style="font-size:15px;">This notebook implements and analyses the Biologically-Inspired Computation coursework, focusing on Artificial Neural Networks (ANNs) and Particle Swarm Optimization (PSO). It includes a multi-layer ANN architecture trained using PSO to solve a regression task predicting concrete compressive strength. Additionally, it explores the impact of hyperparameters on model performance through systematic experiments.</p>

**<p style="font-size:18px;">Data Preparation</p>**
This contains all the data preprocessing and cleaning</p>

In [26]:
import os
print(os.getcwd())

/Users/imanteh/f20bc_new/F21BC/Notebooks


In [27]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import random
from sklearn.preprocessing import LabelEncoder, StandardScaler

In [28]:
df = pd.read_csv("../Dataset/concrete_data.csv", skiprows=1)

In [29]:
df.head()

,cement,blast_furnace_slag,fly_ash,water,superplasticizer,coarse_aggregate,fine_aggregate,age,concrete_compressive_strength
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28,79.99
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28,61.89
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270,40.27
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365,41.05
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360,44.30


In [30]:
# Viewing the dimensionality of the dataset
df.shape

(1030, 9)

In [31]:
df.dropna(inplace = True)
df.isnull().sum()

cement                           0
blast_furnace_slag               0
fly_ash                          0
water                            0
superplasticizer                 0
coarse_aggregate                 0
fine_aggregate                   0
age                              0
concrete_compressive_strength    0
dtype: int64

In [32]:
print(f'Duplicate rows: {df.duplicated().sum()}')

Duplicate rows: 25


In [33]:
# Remove duplicate rows
df.drop_duplicates(inplace=True)

**<p style="font-size:18px;">Scaling and Encoding</p>**

In [34]:
print(df.head())
print(df.columns)

   cement  blast_furnace_slag  fly_ash  water  superplasticizer  \
0   540.0                 0.0      0.0  162.0               2.5   
1   540.0                 0.0      0.0  162.0               2.5   
2   332.5               142.5      0.0  228.0               0.0   
3   332.5               142.5      0.0  228.0               0.0   
4   198.6               132.4      0.0  192.0               0.0   

   coarse_aggregate  fine_aggregate   age  concrete_compressive_strength  
0            1040.0            676.0   28                          79.99  
1            1055.0            676.0   28                          61.89  
2             932.0            594.0  270                          40.27  
3             932.0            594.0  365                          41.05  
4             978.4            825.5  360                          44.30  
Index(['cement', 'blast_furnace_slag', 'fly_ash', 'water', 'superplasticizer',
       'coarse_aggregate', 'fine_aggregate ', 'age',
       'concret

In [35]:
from sklearn.model_selection import train_test_split

X = df.iloc[:, :-1] # All columns except the last
y = df.iloc[:, -1] # The last column

In [36]:
print(X.shape)
print(y.shape)

(1005, 8)
(1005,)


In [37]:
print(X.head())

   cement  blast_furnace_slag  fly_ash  water  superplasticizer  \
0   540.0                 0.0      0.0  162.0               2.5   
1   540.0                 0.0      0.0  162.0               2.5   
2   332.5               142.5      0.0  228.0               0.0   
3   332.5               142.5      0.0  228.0               0.0   
4   198.6               132.4      0.0  192.0               0.0   

   coarse_aggregate  fine_aggregate   age  
0            1040.0            676.0   28  
1            1055.0            676.0   28  
2             932.0            594.0  270  
3             932.0            594.0  365  
4             978.4            825.5  360  


In [38]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Scale features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


In [39]:
print("First 5 rows of scaled training data:")
print(X_train_scaled[:5])

First 5 rows of scaled training data:
[[-0.7036142   0.7362629  -0.8810702   0.1799759  -1.0226942   1.32341753
  -0.17765612 -0.62455318]
 [-0.83112259 -0.84582676  1.04788182 -0.72981642  0.63596651  1.35948653
   0.32102131  0.88513966]
 [ 0.43140366  1.06620495 -0.8810702   0.38479825 -0.18498677 -1.33280675
   0.00745899 -0.28365479]
 [-1.05233032  0.66864226  1.10026551 -0.30588178  0.28412939  0.42298068
  -0.33758549 -0.51092038]
 [-1.17404287 -0.84582676  1.31904441  0.5419875   0.50193332 -1.24005789
   1.18741037 -0.28365479]]


**<p style="font-size:18px;">Artificial Neural Network (ANN)</p>**
Below are the implementations of the ANN to train and predict concrete compressive strength

In [40]:
class activationFunction:
    def logisticFunction(x):
        return 1 / (1 + np.exp(-x))

    def reluFunction(x):
        return np.maximum(0, x)

    def hyperbolicFunction(x):
        return np.tanh(x)

    def leakyReLU(x, alpha=0.01):
        return np.maximum(x, alpha * x)

    def linearFunction(x):  # Identity function for regression output
        return x
class ArtificialNeuralNetwork:
    def __init__(self, layerSize, activationFunction):
        self.layerSize = layerSize
        self.activationFunction = activationFunction
        self.weights = [np.random.randn(layer_sizes[i], layer_sizes[i + 1]) * np.sqrt(2 / layer_sizes[i]) for i in range(len(layer_sizes) - 1)]
        self.biases = [np.random.randn(1, layer_sizes[i + 1]) for i in range(len(layer_sizes) - 1)]

    def visualize_layers(self):
        for i in range(len(self.layerSize)-1):
            print("LAYER COUNT:  ", i,"    NODES: ",self.layerSize[i])
            print(f"  Weights Shape:   ", self.weights[i].shape)
            print(f"  Biases Shape:    ",self.biases[i].shape )
            print(f"  Weights= : {self.weights[i]}")
            print(f"  Biases= {self.biases[i]}")

    def forwardPropagation(self, x):
        output = x
        for i in range(len(self.weights)):
            matrix_total = np.dot(output, self.weights[i]) + self.biases[i]
            output = self.activationFunction[i](matrix_total)
        return output

Layer size is structure to be 8,16,8,1 because
- 8 input layers
- 16 neurons in first hidden layer
- 8 for the second hidden layer
- 1 for the output layer<p>
This is to check the predicted values generated after forward propagation from the ANN.</p>

In [41]:
layer_sizes = [8, 16, 8, 1]

activation_functions = [
    activationFunction.logisticFunction,
    activationFunction.reluFunction,
    activationFunction.linearFunction,
]
ann = ArtificialNeuralNetwork(layer_sizes, activation_functions)
ann.visualize_layers()

x_example = np.random.rand(8, 8)  #8 samples, 8 features
output = ann.forwardPropagation(x_example)
print("ANN Output (Sample Predictions):", output)

LAYER COUNT:   0     NODES:  8
  Weights Shape:    (8, 16)
  Biases Shape:     (1, 16)
  Weights= : [[ 0.29367945  0.74666871 -0.51852418  0.15795627  0.25641905 -0.06810522
   1.10132209 -1.17184619  0.72483216  0.27661611 -0.17049527 -0.13834417
  -0.80000271 -0.46849772 -0.31123999  0.51878735]
 [-0.17259388 -0.16678556 -0.65534467 -1.01676702  0.28647038  0.29824349
  -0.00276215  0.08746842 -0.28824454 -0.36494868 -0.19725076 -0.02831527
  -0.03613549 -0.85286754 -0.0641186   0.46606537]
 [-0.44093624 -0.20640608 -0.89098179 -0.32068325 -0.65988784  1.32328379
  -0.21625058  0.17107191  1.00250763 -0.66028855 -0.54536065  0.02975734
  -0.51597536  0.57716791  0.32726788 -0.55816624]
 [ 0.00596332  0.43833361  0.35562912  0.6162114  -0.26981013  0.74866949
  -0.34486818  0.00920199  0.51463024  0.28085785  0.59645865 -0.02718709
   0.74889338 -0.03710387  0.25563561  0.74387914]
 [ 0.48966827  0.18730209 -0.38720922  0.28773436  0.16701873 -0.50654748
  -0.18179284  0.89438166 -0.5

**<p style="font-size:18px;">Loss Function</p>**
Since the problem domian is regression, MAE is utilised as the loss function

In [42]:
class lossFunction:
    def evaluate(self,y_pred,y_train):
        self.y_pred=y_pred
        self.y_train=y_train
class MeanAbsoluteError(lossFunction):
    def evaluate(self, y_pred, y_train):
        return np.mean(np.abs(y_pred - y_train))

In [43]:
y_train = y_train.to_numpy().reshape(-1, 1)
y_pred = ann.forwardPropagation(X_train_scaled) # Forward propagating  through the ANN
# checking y_pred and y_train has the correct shape
print("y_pred shape:", y_pred.shape)
print("y_train shape:", y_train.shape)

# use to calculate the loss function
loss_function=MeanAbsoluteError()
loss=loss_function.evaluate(y_pred, y_train)
print("Loss:", loss)

y_pred shape: (703, 1)
y_train shape: (703, 1)
Loss: 35.44828747629815


**<p style="font-size:18px;">Implement PSO Algorithm</p>**

In [44]:
class Particle:
    def __init__(self,vectorSize):
            self.particlePosition=np.random.rand(vectorSize)
            self.particleVelocity=np.random.rand(vectorSize)
            self.bestPosition=np.copy(self.particlePosition)
            self.informants=[]

In [45]:
def particleToAnn(particle, annLayers, activationFunctions):
    neuralNetwork = ArtificialNeuralNetwork(layerSize=annLayers, activationFunction=activationFunctions)
    weightBiasIndexCount = 0
    for i in range(len(annLayers) - 1):
        prevValue = annLayers[i]
        nextValue = annLayers[i + 1]
        weightRange = prevValue * nextValue

        weight = particle.particlePosition[weightBiasIndexCount:weightBiasIndexCount + weightRange].reshape((prevValue, nextValue))
        weightBiasIndexCount += weightRange

        biases = particle.particlePosition[weightBiasIndexCount:weightBiasIndexCount + nextValue].reshape((1, nextValue))
        weightBiasIndexCount += nextValue

        activation = activationFunctions[i]
        neuralNetwork.weights[i] = weight
        neuralNetwork.biases[i] = biases
    return neuralNetwork

In [46]:
def assessFitness(particle, dataset, annLayers, activationFunctions, loss_function):
    x, y = dataset
    ann = particleToAnn(particle, annLayers, activationFunctions)
    predictions = ann.forwardPropagation(x)
    loss = loss_function.evaluate(predictions, y.reshape(-1, 1))
    return loss

In [47]:
class ParticleSwarmOptimisation:
    def __init__(self, swarmSize, alpha, beta, delta, gamma, jumpSize, informantCount, vectorSize):
        self.swarmSize = swarmSize
        self.alpha = alpha
        self.beta = beta
        self.delta = delta
        self.gamma =gamma
        self.jumpSize = jumpSize
        self.informantCount = informantCount
        self.vectorSize = vectorSize
        self.global_best = None
        self.global_best_fitness = float('inf')

    def initInformants(self, informantCount, particleArray):
        for p in particleArray:
            potentialInformants = [potInf for potInf in particleArray if potInf != p]
            p.informants = [random.choice(potentialInformants) for _ in range(informantCount)]

    def get_best_informant(self, particle, dataset, annLayers, activationFunctions, loss_function):
        bestInf = None
        bestFitnessInf = float('-inf')
        for i in particle.informants:
            fitness = assessFitness(i, dataset, annLayers, activationFunctions, loss_function)
            if fitness > bestFitnessInf:
                bestFitnessInf = fitness
                bestInf = i
        return bestInf.particlePosition

    def psoOptimisation(self, swarmSize, alpha, beta, gamma, jumpSize, informantCount, vectorSize,
                        dataset, annLayers, activationFunctions, loss_function, max_iterations=100):

        particleArray = [Particle(vectorSize) for _ in range(swarmSize)]
        self.initInformants(informantCount, particleArray)

        best = None
        iteration = 0

        while iteration < max_iterations:
            # Update global best
            for p in particleArray:
                particleFitness = assessFitness(p, dataset, annLayers, activationFunctions, loss_function)
                if best is None or particleFitness < assessFitness(best, dataset, annLayers, activationFunctions, loss_function):
                    best = p

            for p in particleArray:
                previousBest = p.bestPosition
                informantsBest = self.get_best_informant(p, dataset, annLayers, activationFunctions, loss_function)
                allBest = best.bestPosition

                b = np.random.uniform(0.0, beta)
                c = np.random.uniform(0.0, gamma)
                d = np.random.uniform(0.0, delta)

                updatedVelocity = (
                    alpha * p.particleVelocity +
                    b * (previousBest - p.particlePosition) +
                    c * (informantsBest - p.particlePosition) +
                    d * (allBest - p.particlePosition)
                )
                print("updated v:", updatedVelocity)
                p.particleVelocity = updatedVelocity
                p.particlePosition += jumpSize * updatedVelocity

            iteration += 1

        return best.particlePosition

<p style="font-size:18px;">Test out PSO Functionality</p>

In [48]:
# Code to test out PSO functionality
X_train_scaled = np.random.rand(10, 8)  # Simulated scaled training features (100 samples, 8 features)
y_train = np.random.rand(10, 1)  # Simulated training labels (100 samples, 1 output)

# define hyperparameters for PSO
swarmSize = 10
alpha = 0.9  # inertia weight
beta = 0.7   # cognitive parameter
gamma = 0.5  # social parameter
delta = 0.5
jumpSize = 0.5
informantCount = 2
annLayers=layer_sizes
vectorSize = sum([(annLayers[i] * annLayers[i + 1]) + annLayers[i + 1] for i in range(len(annLayers) - 1)])  # Total weights + biases
max_iterations = 10

# initialize the PSO optimiser
pso = ParticleSwarmOptimisation(
    swarmSize = swarmSize,
    alpha = alpha,
    beta = beta,
    delta = delta,
    gamma = gamma,
    jumpSize = jumpSize,
    informantCount = informantCount,
    vectorSize = vectorSize
)

# run it into the psoOptimisation
best_parameters = pso.psoOptimisation(
    swarmSize = swarmSize,
    alpha = alpha,
    beta = beta,
    gamma = delta,  # Using delta as gamma as per your code
    jumpSize = jumpSize,
    informantCount = informantCount,
    vectorSize = vectorSize,

    dataset = (X_train_scaled, y_train),
    annLayers = annLayers,
    activationFunctions = activation_functions,
    loss_function= MeanAbsoluteError(),
    max_iterations = max_iterations
)
print("Optimised ANN Parameters:", best_parameters)

updated v: [ 0.2337418   0.18372653  0.16574347  0.15272285  0.52224626  1.02795882
  0.2974044   0.77214764  0.22061566  0.08676216  1.33823052  0.6585437
  0.1132942   0.56782126  0.51967642 -0.16879021  0.77611974 -0.25120348
  0.4625865   1.20449769  0.54921546  0.02853139  0.16185478  0.82417372
  0.65310938  0.19247278  0.47222985  1.10771233  0.25261803 -0.27718541
  0.08833732 -0.1756801   0.05045634  0.09166275  0.4267114   0.6619284
  0.62326505  0.4463095   0.57899138  1.17918126  0.29184556  0.45482923
  0.48561404  1.04054332  0.32197734  0.9588467  -0.16215183  0.63997293
  0.73097333  0.28307023  0.97700701  0.96665415 -0.15030244  0.45268849
  0.18982088  0.29556489  0.08089647  0.17331303  0.86159581 -0.10313349
  0.95186591  0.5448877   0.41400768  0.40683813  0.32139023  0.90155331
  0.26742795  0.68008607  0.57611496 -0.14810831  0.23235644  0.54929397
  0.55822247  0.44021463  0.31090861  0.57962693  0.26171948  1.03114927
  0.81291623  0.43993132  0.20951147  0.08

<p style="font-size:18px;">Get Fitness Value</p>
The final fitness represents the performance of the ANN after its weights and biases have been optimised by the PSO

In [49]:
# create a dummy particle and assign the optimised parameters
optimised_particle = Particle(vectorSize)
optimised_particle.particlePosition = best_parameters

# convert the optimised particle position back to ANN weights and biases
optimised_ann = particleToAnn(
    optimised_particle,
    annLayers,
    activation_functions
)

# run through ANN with the optimised parameters
fitness_value = assessFitness(
    optimised_particle,  # pass the optimised particle for fitness evaluation
    dataset = (X_train_scaled, y_train),
    annLayers = annLayers,
    activationFunctions = activation_functions,
    loss_function = MeanAbsoluteError()
)
print("Fitness Value:", fitness_value)

Fitness Value: 3.60329164128877


<p style="font-size:18px;">Training the ANN</p>

In [25]:
# Evaluate ANN with optimized parameters on the test set
def evaluate_ann_with_optimized_parameters(best_parameters, X_test_scaled, y_test, annLayers, activation_functions):
    # Create a dummy particle with the optimized parameters
    optimized_particle = Particle(vectorSize=len(best_parameters))
    optimized_particle.particlePosition = best_parameters
    
    # Convert the particle's position to ANN weights and biases
    optimized_ann = particleToAnn(
        optimized_particle, 
        annLayers, 
        activation_functions
    )
    
    # Forward propagate the test data through the ANN
    predictions = optimized_ann.forwardPropagation(X_test_scaled)
    
    # Convert y_test to a NumPy array and reshape it
    y_test_array = y_test.to_numpy().reshape(-1, 1)
    
    # Calculate the MAE using the test dataset
    loss_function = MeanAbsoluteError()
    test_loss = loss_function.evaluate(predictions, y_test_array)
    
    print(f"Test MAE with optimized parameters: {test_loss}")

evaluate_ann_with_optimized_parameters(
    best_parameters=best_parameters,
    X_test_scaled=X_test_scaled,
    y_test=y_test,
    annLayers=layer_sizes, # Replace with your ANN architecture
    activation_functions=activation_functions
)


Test MAE with optimized parameters: 28.12755318620683
